In [2]:
import pandas as pd
import plotly 
import numpy as np
import plotly.figure_factory as ff
import scipy
import plotly.graph_objects as go
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.express as px

#pip install scipy
#!pip install Dash

In [3]:
# Import data

data = pd.read_csv('chocolate.csv')
continent = pd.read_csv('countryContinent.csv',encoding = "ISO-8859-1")
imp_exp=pd.read_csv('UNdata_Export_20220301_151116452.csv')
coord=pd.read_csv('country_points.csv',encoding = "ISO-8859-1")

In [4]:
imp_exp=imp_exp[imp_exp['Commodity']=='Cocoa beans, whole or broken, raw or roasted'][['Country or Area','Year','Commodity','Flow','Quantity','Trade (USD)']]
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


### Exports and Imports


In [5]:
imp_exp_regions = imp_exp.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'sub_region':'region'}), left_on = 'Country or Area', right_on= 'country', how= 'left')

In [6]:
imp_exp_regions.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2
0,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0,Albania,Europe,Southern Europe,AL
1,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0,Albania,Europe,Southern Europe,AL
2,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0,Albania,Europe,Southern Europe,AL
3,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0,Albania,Europe,Southern Europe,AL
4,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0,Albania,Europe,Southern Europe,AL


export = imp_exp_regions[imp_exp_regions['Flow'] == 'Export']
export.head()

export[export['continent'] == 'Europe']['Country or Area'].unique()

imports = imp_exp_regions[imp_exp_regions['Flow'] == 'Import']
imports.head()

In [7]:
pivot_imports = pd.pivot_table(imports, values = 'Trade (USD)', index = 'continent', columns = 'Year', aggfunc = np.sum)

NameError: name 'imports' is not defined

In [ ]:
pivot_imports.index.name = None
pivot_imports

In [ ]:
pivot_export = pd.pivot_table(export, values = 'Trade (USD)', index = 'continent', columns = 'Year', aggfunc = np.sum)

In [ ]:
pivot_export.index.name = None
pivot_export.index

In [ ]:
#frame1 = []
#frame2 = []

#frames = []

#for year in pivot_export.columns[1:]: # we do not include the first frame here
 #   frames.append(dict(data=dict(type='bar',
  #                               x=pivot_export.index,
   #                              y=pivot_export[year],
    #                             name=year,
     #                           ),
      #                 layout=go.Layout(title_text='Exports in the year of ' + str(year))
       #               )
        #        )

#for year in pivot_imports.columns[1:]: # we do not include the first frame here
#    frames.append(dict(data=dict(type='scatter',
#                                 x=pivot_imports.index,
#                                 y=pivot_imports[year],
#                                 name=year,
#                                ),
#                       layout=go.Layout(title_text='Imports in the year of ' + str(year))
#                      )
#                )

    
#frames = [frame1, frame2]

# First introduction to buttons
#fig_bar_layout = dict(title=dict(text='Exports by continent from 1988 to 2019'),
 #                     yaxis=dict(title='Exports',
  #                               range=[0,9000000000]
   #                             ),
    #                  updatemenus=[dict(type="buttons",
     #                               buttons=[dict(label="Play",
      #                                            method="animate",
       #                                           args=[None])
        #                                    ]
         #                              )
           #                       ]
            #        )

#initial_data = dict(type='bar', 
 #                   x=pivot_export.index, 
  #                  y=pivot_export.values[0],
   #                 name=str(1988))

#fig_bar = go.Figure(data = initial_data, layout = fig_bar_layout, frames=frames)

#fig_bar.show(renderer='iframe')

### ------------- BEA


In [8]:
imp_exp_regions.loc[(imp_exp_regions['continent']=='nan') & (imp_exp_regions['Country or Area']!='nan')]

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2


In [9]:
imp_exp_regions['country'].isna().sum()

648

In [10]:
imp_exp_regions['continent'].isna().sum()

648

In [11]:
imp_exp_regions= imp_exp_regions[~(imp_exp_regions['Flow'] == 'Re-Export')]
imp_exp_regions= imp_exp_regions[~(imp_exp_regions['Flow'] == 'Re-Import')]

In [12]:
#imp_exp_regions[imp_exp_regions['Quantity'].isna()==True]

In [13]:
imp_exp_regions["Quantity"]= imp_exp_regions["Quantity"].fillna(imp_exp_regions.groupby('Country or Area')['Quantity'].transform('mean'))

In [14]:
imp_exp_regions['Quantity'].astype(int)

0         130
1        4524
2          45
3         262
4        1606
        ...  
5053       79
5054      221
5055      934
5056    14942
5057        9
Name: Quantity, Length: 4898, dtype: int32

In [15]:
imp_exp_regions=imp_exp_regions[imp_exp_regions['Country or Area'].isnull()!= True] #removing nulls from Country or Area Column

## Se necessário, mudar aqui:
imports.loc[imports['Country or Area'] == 'Wallis and Futuna Isds', 'continent'] = 'Oceania'
imports.loc[imports['Country or Area'] == 'Wallis and Futuna Isds','region'] = 'Polynesia'
#imports.loc[imports['Country or Area'] == 'United Rep. of Tanzania','country'] = 'Tanzania'

In [16]:
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Wallis and Futuna Isds', 'continent'] = 'Oceania'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Wallis and Futuna Isds','region'] = 'Polynesia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Venezuela', 'continent'] = 'Americas'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Venezuela','region'] = 'Southern America'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'United Rep. of Tanzania', 'continent'] = 'Africa'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'United Rep. of Tanzania','region'] = 'Eastern Africa'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'United Rep. of Tanzania','country'] = 'Tanzania'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'United Kingdom', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'United Kingdom','region'] = 'Northern Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'USA', 'continent'] = 'Americas'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'USA','region'] = 'Northern America'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'USA','country'] = 'United States of America'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Turks and Caicos Isds', 'continent'] = 'Americas'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Turks and Caicos Isds','region'] = 'Central America'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'TFYR of Macedonia', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'TFYR of Macedonia','region'] = 'Southern Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'TFYR of Macedonia','country'] = 'Macedonia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Syria', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Syria','region'] = 'Western Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Solomon Isds', 'continent'] = 'Oceania'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Solomon Isds','region'] = 'Australia and New Zealand'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Serbia and Montenegro', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Serbia and Montenegro','region'] = 'Southern Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Rep. of Moldova', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Rep. of Moldova','region'] = 'Eastern Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Rep. of Moldova','country'] = 'Moldova'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Rep. of Korea', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Rep. of Korea','region'] = 'Eastern Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Rep. of Korea','country'] = 'South Korea'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Other Asia, nes', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Other Asia, nes','region'] = 'Other'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Neth. Antilles', 'continent'] = 'Americas'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Neth. Antilles','region'] = 'Caribbean'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Lao People\'s Dem. Rep.', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Lao People\'s Dem. Rep.','region'] = 'South-Eastern Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Lao People\'s Dem. Rep.','country'] = 'Laos'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Iran', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Iran','region'] = 'South-Western Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Fmr Fed. Rep. of Germany', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Fmr Fed. Rep. of Germany','region'] = 'Central Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Fmr Fed. Rep. of Germany','country'] = 'Germany'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Faeroe Isds', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Faeroe Isds', 'region'] = 'Northern Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'FS Micronesia', 'continent'] = 'Oceania'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'FS Micronesia', 'region'] = 'Micronesia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Czech Rep.', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Czech Rep.', 'region'] = 'Central Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Bosnia Herzegovina', 'continent'] = 'Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Bosnia Herzegovina', 'region'] = 'Southern Europe'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Central African Rep.', 'continent'] = 'Africa'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Central African Rep.', 'region'] = 'Middle Africa'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'China, Hong Kong SAR', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'China, Hong Kong SAR', 'region'] = 'Eastern Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'China, Macao SAR', 'continent'] = 'Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'China, Macao SAR', 'region'] = 'Eastern Asia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Cook Isds', 'continent'] = 'Oceania'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Cook Isds', 'region'] = 'Polynesia'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Dominican Rep.', 'continent'] = 'Americas'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Dominican Rep.', 'region'] = 'Caribbean'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Fmr Sudan', 'continent'] = 'Africa'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Fmr Sudan','region'] = 'Northern Africa'
imp_exp_regions.loc[imp_exp_regions['Country or Area'] == 'Fmr Sudan','country'] = 'Sudan'

In [17]:
imp_exp_regions= imp_exp_regions[~(imp_exp_regions['Country or Area'] == 'Belgium-Luxembourg')]

In [18]:
imp_exp_regions= imp_exp_regions[~(imp_exp_regions['Country or Area'] == 'EU-28')]
imp_exp_regions= imp_exp_regions[~(imp_exp_regions['Country or Area'] == 'So. African Customs Union')]
imp_exp_regions= imp_exp_regions[~(imp_exp_regions['Country or Area'] == 'China, Hong Kong SAR')]


In [19]:
flows_df=imp_exp_regions.drop(columns=['country', 'code_2'])

# Visualization 

In [20]:
#convert data type and sort the data by Year
flows_df=flows_df.sort_values(by=['Year'])

In [21]:
flows_df=flows_df.dropna(how='any')

In [22]:
min(flows_df['Quantity'])

0.0

In [73]:
groupby_flows=flows_df.groupby(['Year','continent', 'region', 'Country or Area', 'Flow']).mean(['Trade (USD)','Quantity'])
groupby_flows.reset_index(inplace=True)
groupby_flows

,Year,continent,region,Country or Area,Flow,Quantity,Trade (USD)
0,1991,Africa,Eastern Africa,Madagascar,Export,4.272629e+06,3.971947e+06
1,1991,Africa,Eastern Africa,Malawi,Import,1.010000e+02,2.920000e+02
2,1991,Africa,Northern Africa,Tunisia,Import,3.555000e+05,5.718130e+05
3,1991,Americas,Caribbean,Haiti,Export,1.395671e+06,1.092508e+06
4,1991,Americas,Caribbean,Jamaica,Export,1.481214e+06,2.234101e+06
...,...,...,...,...,...,...,...
4717,2019,Europe,Southern Europe,Greece,Import,3.279127e+06,8.264908e+06
4718,2019,Europe,Southern Europe,TFYR of Macedonia,Export,3.000000e+00,4.538000e+01
4719,2019,Europe,Southern Europe,TFYR of Macedonia,Import,4.660000e+02,4.885310e+03
4720,2019,Europe,Western Europe,Switzerland,Export,3.428100e+05,1.724814e+06


In [74]:
groupby_flows.reset_index(inplace=True)

In [75]:
groupby_flows=groupby_flows[groupby_flows['Year']!=1988]
groupby_flows=groupby_flows[groupby_flows['Year']!=1989]
groupby_flows=groupby_flows[groupby_flows['Year']!=1990]
#delete years until all continents have value for exports and imports


In [76]:
flows_df=flows_df[flows_df['Year']!=1988]
flows_df=flows_df[flows_df['Year']!=1989]
flows_df=flows_df[flows_df['Year']!=1990]

In [77]:
groupby_flows

,index,Year,continent,region,Country or Area,Flow,Quantity,Trade (USD)
0,0,1991,Africa,Eastern Africa,Madagascar,Export,4.272629e+06,3.971947e+06
1,1,1991,Africa,Eastern Africa,Malawi,Import,1.010000e+02,2.920000e+02
2,2,1991,Africa,Northern Africa,Tunisia,Import,3.555000e+05,5.718130e+05
3,3,1991,Americas,Caribbean,Haiti,Export,1.395671e+06,1.092508e+06
4,4,1991,Americas,Caribbean,Jamaica,Export,1.481214e+06,2.234101e+06
...,...,...,...,...,...,...,...,...
4717,4717,2019,Europe,Southern Europe,Greece,Import,3.279127e+06,8.264908e+06
4718,4718,2019,Europe,Southern Europe,TFYR of Macedonia,Export,3.000000e+00,4.538000e+01
4719,4719,2019,Europe,Southern Europe,TFYR of Macedonia,Import,4.660000e+02,4.885310e+03
4720,4720,2019,Europe,Western Europe,Switzerland,Export,3.428100e+05,1.724814e+06


In [78]:
africa_count= len(flows_df.groupby('continent')['Country or Area'].unique()[0])
americas_count= len(flows_df.groupby('continent')['Country or Area'].unique()[1])
asia_count= len(flows_df.groupby('continent')['Country or Area'].unique()[2])
europe_count= len(flows_df.groupby('continent')['Country or Area'].unique()[3])
oceania_count= len(flows_df.groupby('continent')['Country or Area'].unique()[4])

In [79]:
print('Africa has: ' + str(africa_count) + ' countries')
print('Americas has: ' + str(americas_count) + ' countries')
print('Asia has: ' + str(asia_count) + ' countries')
print('Europe has: ' + str(europe_count) + ' countries')
print('Oceania has: ' + str(oceania_count) + ' countries')

Africa has: 49 countries
Americas has: 43 countries
Asia has: 44 countries
Europe has: 42 countries
Oceania has: 15 countries


In [80]:
nr_countries = [['Africa', africa_count],
                ['Americas', americas_count], 
                ['Asia', asia_count],
                ['Europe', europe_count],
                ['Oceania', oceania_count]]

df_nr_countries = pd.DataFrame(nr_countries, columns = ['Continent', 'Nr of Countries'])
print(df_nr_countries)

  Continent  Nr of Countries
0    Africa               49
1  Americas               43
2      Asia               44
3    Europe               42
4   Oceania               15


In [81]:
groupby_flows= pd.merge(df_nr_countries, groupby_flows, left_on='Continent', right_on='continent')
groupby_flows.drop(columns='continent', inplace=True)
groupby_flows

,Continent,Nr of Countries,index,Year,region,Country or Area,Flow,Quantity,Trade (USD)
0,Africa,49,0,1991,Eastern Africa,Madagascar,Export,4272629.0,3.971947e+06
1,Africa,49,1,1991,Eastern Africa,Malawi,Import,101.0,2.920000e+02
2,Africa,49,2,1991,Northern Africa,Tunisia,Import,355500.0,5.718130e+05
3,Africa,49,49,1992,Eastern Africa,Kenya,Export,810.0,1.379000e+03
4,Africa,49,50,1992,Eastern Africa,Kenya,Import,342375.0,5.445820e+05
...,...,...,...,...,...,...,...,...,...
4717,Oceania,15,4690,2018,Melanesia,Fiji,Export,8270.0,2.248422e+04
4718,Oceania,15,4691,2018,Melanesia,Fiji,Import,528.0,3.499561e+03
4719,Oceania,15,4692,2018,Micronesia,Palau,Import,4.5,5.595000e+01
4720,Oceania,15,4693,2018,Polynesia,Samoa,Export,34976.6,2.608072e+05


In [82]:
#groupby_flows['Q_avg_pond']=groupby_flows.apply(lambda row: row['Quantity']/row['Nr of Countries'], axis=1)
#groupby_flows['T_avg_pond']=groupby_flows.apply(lambda row: row['Trade (USD)']/row['Nr of Countries'], axis=1)
#TO TEST VISUALIZATIONS WITH MEDIA PONDERADA - not used so far

In [83]:
groupby_flows

,Continent,Nr of Countries,index,Year,region,Country or Area,Flow,Quantity,Trade (USD)
0,Africa,49,0,1991,Eastern Africa,Madagascar,Export,4272629.0,3.971947e+06
1,Africa,49,1,1991,Eastern Africa,Malawi,Import,101.0,2.920000e+02
2,Africa,49,2,1991,Northern Africa,Tunisia,Import,355500.0,5.718130e+05
3,Africa,49,49,1992,Eastern Africa,Kenya,Export,810.0,1.379000e+03
4,Africa,49,50,1992,Eastern Africa,Kenya,Import,342375.0,5.445820e+05
...,...,...,...,...,...,...,...,...,...
4717,Oceania,15,4690,2018,Melanesia,Fiji,Export,8270.0,2.248422e+04
4718,Oceania,15,4691,2018,Melanesia,Fiji,Import,528.0,3.499561e+03
4719,Oceania,15,4692,2018,Micronesia,Palau,Import,4.5,5.595000e+01
4720,Oceania,15,4693,2018,Polynesia,Samoa,Export,34976.6,2.608072e+05


In [84]:
groupby_flows = groupby_flows.sort_values('Year', ascending=True)

In [85]:
def scatter_flows(flow):
    fig = px.scatter(groupby_flows[groupby_flows['Flow']==flow],
                 x='Quantity',
                 y='Trade (USD)',
                 animation_frame='Year', 
                 animation_group='Continent', 
                 size='Nr of Countries', 
                 color='Continent',
                 hover_name='Continent',
                 )
    fig.update_layout(
                    title=('Quantity of Cacao ' + str(flow) + 'ed vs Trade (USD) by Continents in 1988 - 2019'),
                    xaxis=dict(title=('Quantity '+ str(flow) + 'ed')),
                    yaxis=dict(title=(str(flow) + ' Trade (USD) of Cacao')),
                    #paper_bgcolor='rgb(500, 500, 500)'#,
                    #plot_bgcolor='rgb(243, 243, 243)'
                    )
                    
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 3000
    return fig.show()

In [86]:
scatter_flows(flow='Import')

In [87]:
scatter_flows(flow='Export')

In [45]:
n_years= 2019-1988
n_years

31

In [90]:
tree_flows_GB=groupby_flows.groupby(['Flow','Continent','region', 'Country or Area']).mean(['Quantity','Trade (USD)'])
tree_flows_GB.reset_index(inplace=True)
tree_flows_GB

,Flow,Continent,region,Country or Area,Nr of Countries,index,Year,Quantity,Trade (USD)
0,Export,Africa,Eastern Africa,Kenya,49.0,1964.500000,2004.857143,2.009929e+03,1.265276e+03
1,Export,Africa,Eastern Africa,Madagascar,49.0,1953.642857,2004.500000,5.113197e+06,9.890885e+06
2,Export,Africa,Eastern Africa,Malawi,49.0,2558.666667,2008.333333,3.991800e+05,3.096376e+06
3,Export,Africa,Eastern Africa,Mozambique,49.0,1538.000000,2003.000000,2.000000e+00,5.540000e+02
4,Export,Africa,Eastern Africa,Rwanda,49.0,4095.000000,2016.000000,2.675000e+03,7.440000e+03
...,...,...,...,...,...,...,...,...,...
335,Import,Oceania,Polynesia,Cook Isds,15.0,2675.000000,2008.000000,1.750000e+01,3.550000e+01
336,Import,Oceania,Polynesia,French Polynesia,15.0,2886.666667,2009.000000,8.308333e+01,8.285833e+02
337,Import,Oceania,Polynesia,Samoa,15.0,3347.111111,2011.222222,2.555556e+03,5.615024e+03
338,Import,Oceania,Polynesia,Tonga,15.0,2302.384615,2006.000000,1.051154e+03,2.947154e+03


In [91]:
def tree_flows(flow, variable):
    fig = px.treemap(tree_flows_GB[tree_flows_GB['Flow']==flow], 
                     path=['Continent', 'region', 'Country or Area'],
                     values=variable, color=variable, color_continuous_scale='sunset'

                    )
                
    return fig.show()

In [93]:
tree_flows(flow='Import', variable='Quantity')